In [1]:
from typing import TYPE_CHECKING


if TYPE_CHECKING:
    from math_rag.application.containers import ApplicationContainer
    from math_rag.infrastructure.containers import InfrastructureContainer

    application_container: ApplicationContainer
    infrastructure_container: InfrastructureContainer

In [2]:
RESET = False
%load_ext hooks.notebook_hook

2025-06-21 18:11:10,932 - INFO - PyTorch version 2.6.0 available.


In [3]:
importer_service = application_container.math_expression_label_task_importer_service()
exporter_service = application_container.math_expression_label_exporter_service()

result_repository = infrastructure_container.math_expression_dataset_test_result_repository()
label_repository = infrastructure_container.math_expression_label_repository()

In [4]:
from uuid import UUID


dataset_id = UUID('7cdbb987-0994-4d59-8d9c-5e827e000769')
split_name = 'test'

In [5]:
project_id = await importer_service.import_tasks(None, dataset_id=dataset_id, split_name=split_name)

README.md:   0%|          | 0.00/4.04k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.17k [00:00<?, ?B/s]

validate-00000-of-00001.parquet:   0%|          | 0.00/6.49k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/24.9k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/27 [00:00<?, ? examples/s]

Generating validate split:   0%|          | 0/27 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/221 [00:00<?, ? examples/s]

2025-06-21 13:46:32,476 - INFO - Project math_expression_label_task | 7cdbb987 | test created
2025-06-21 13:46:32,531 - INFO - Imported 221 tasks into math_expression_label_task | 7cdbb987 | test


In [4]:
await infrastructure_container.llm_failed_request_repository().find_many()

ModuleNotFoundError: No module named 'math_rag.application.models.assistants.base.BaseAssistantOutput'

In [10]:
from uuid import uuid4

from math_rag.application.models.assistants import KatexCorrectorAssistantOutput


output = KatexCorrectorAssistantOutput.bind(uuid4())
output

math_rag.application.models.assistants.base.base_assistant_output.BaseAssistantOutput.bind.<locals>.BoundAssistantOutput

In [ ]:
dataset_labels = await label_repository.find_many(
    filter=dict(math_expression_dataset_id=dataset_id, math_expression_id=data)
)

In [7]:
human_labels = await exporter_service.export(project_id)

In [40]:
results = await result_repository.find_many(
    filter=dict(
        math_expression_dataset_id=dataset_id, math_expression_dataset_split_name=split_name
    )
)

# llama_labels = results[0].math_expression_labels
# gpt_4_1_labels = results[1].math_expression_labels
gpt_4_1_mini_labels = results[0].math_expression_labels
gpt_4_1_labels = results[1].math_expression_labels

In [16]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from math_rag.core.enums import MathExpressionLabelEnum
from math_rag.core.models import MathExpressionLabel

In [33]:
def _sort_by_math_expression_id(labels: list[MathExpressionLabel]) -> list[MathExpressionLabel]:
    return sorted(
        labels,
        key=lambda label: label.math_expression_id,
    )


def prepare(math_expression_labels: list[MathExpressionLabel]) -> list[str]:
    sorted = _sort_by_math_expression_id(math_expression_labels)

    return [label.value.value for label in sorted]


def evaluate_multiclass_labels(
    y_true: list[str],
    y_pred: list[str],
    labels: list[str],
):
    acc = accuracy_score(y_true, y_pred)
    report = classification_report(
        y_true,
        y_pred,
        labels=labels,
        target_names=labels,
        zero_division=0,
    )
    cm = confusion_matrix(
        y_true,
        y_pred,
        labels=labels,
    )

    print(acc)
    print(report)
    print(cm)

In [29]:
labels = [e.value for e in MathExpressionLabelEnum]

In [42]:
y_true = prepare(human_labels)
y_pred = prepare(gpt_4_1_labels)

evaluate_multiclass_labels(y_true, y_pred, labels)

0.8235294117647058
              precision    recall  f1-score   support

    equality       1.00      1.00      1.00        42
  inequality       1.00      1.00      1.00         7
    constant       0.32      1.00      0.49        17
    variable       1.00      0.86      0.93        22
       other       0.97      0.73      0.83       133

    accuracy                           0.82       221
   macro avg       0.86      0.92      0.85       221
weighted avg       0.93      0.82      0.85       221

[[42  0  0  0  0]
 [ 0  7  0  0  0]
 [ 0  0 17  0  0]
 [ 0  0  0 19  3]
 [ 0  0 36  0 97]]


## WIP


In [ ]:
application_container.katex_corrector_assistant